# Project 1: Higgs Boson Detection

In [2]:
import numpy as np
import csv

## Loading files

In [9]:
with open('test.csv') as csv_file_test:
    test_reader=csv.reader(csv_file_test,delimiter=',')
    test_list=[]
    for row in test_reader:
        test_list.append(row)
    y_test=[]
    x_test_list=[]
    for test_row in test_list:
        y_test.append(test_row[1])
        test_row.pop(1)
        x_test_list.append(test_row)
    x_test=np.array(x_test_list)

In [10]:
x_test.shape

(568239, 31)

In [10]:
def load_data(train_path,test_path):
    train_reader=np.genfromtxt(train_path,delimiter=',',missing_values=-999)
    train_list=[]
    for row in train_reader:
        train_list.append(row)
    y_train_l=[]
    x_train_l=[]
    for train_row in train_list:
        train_row=np.ndarray.tolist(train_row)
        y_train_l.append(train_row[1])
        train_row.pop(1)
        x_train_l.append(train_row)
    Data_structure_X=x_train_l[0]
    x_train_l.pop(0)
    y_train_l.pop(0)
    x_train=np.array(x_train_l)
    y_train=np.array(y_train_l)
    test_reader=np.genfromtxt(test_path,delimiter=',')
    test_list=[]
    for row in test_reader:
        test_list.append(row)
    y_test_l=[]
    x_test_l=[]
    for test_row in test_list:
        test_row=np.ndarray.tolist(test_row)
        y_test_l.append(test_row[1])
        test_row.pop(1)
        x_test_l.append(test_row)
    x_test_l.pop(0)
    y_test_l.pop(0)
    x_test=np.array(x_test_l)
    y_test=np.array(y_test_l)
    return x_train,y_train,x_test,y_test,Data_structure_X
        

In [11]:
 x_train,y_train,x_test,y_test,Data_structure_X=load_data('train.csv','test.csv')

In [16]:
x_train.shape
y_train.shape
print(x_train[3],y_train[3])

[  1.00003000e+05   1.43905000e+02   8.14170000e+01   8.09430000e+01
   4.14000000e-01  -9.99000000e+02  -9.99000000e+02  -9.99000000e+02
   3.31000000e+00   4.14000000e-01   7.59680000e+01   2.35400000e+00
  -1.28500000e+00  -9.99000000e+02   2.26470000e+01  -1.65500000e+00
   1.00000000e-02   5.33210000e+01  -5.22000000e-01  -3.10000000e+00
   3.10820000e+01   6.00000000e-02   8.60620000e+01   0.00000000e+00
  -9.99000000e+02  -9.99000000e+02  -9.99000000e+02  -9.99000000e+02
  -9.99000000e+02  -9.99000000e+02   0.00000000e+00] nan


In [5]:
? np.genfromtxt()